In [1]:
# Load dataset
import netCDF4 as nc

# Open the file
ds = nc.Dataset('./power_901_annual_meteorology_utc.nc.nc4')

In [2]:
# Print the variables and dimensions in the file
print(ds.variables)

{'time': <class 'netCDF4._netCDF4.Variable'>
int32 time(time)
    units: years since 1981-12-31 00:00:00
    long_name: time
unlimited dimensions: 
current shape = (2,)
filling on, default _FillValue of -2147483647 used, 'lat': <class 'netCDF4._netCDF4.Variable'>
float64 lat(lat)
    _FillValue: -999.0
    long_name: latitude
    standard_name: latitude
    units: degrees_north
unlimited dimensions: 
current shape = (361,)
filling on, 'lon': <class 'netCDF4._netCDF4.Variable'>
float64 lon(lon)
    _FillValue: -999.0
    long_name: longitude
    standard_name: longitude
    units: degrees_east
unlimited dimensions: 
current shape = (576,)
filling on, 'RH2M': <class 'netCDF4._netCDF4.Variable'>
float64 RH2M(time, lat, lon)
    _FillValue: -999.0
    long_name: Relative Humidity at 2 Meters
    standard_name: Relative_Humidity_at_2_Meters
    units: %
    valid_max: 100.0
    valid_min: 0.0
    valid_range: [  0. 100.]
unlimited dimensions: 
current shape = (2, 361, 576)
filling on, 'T2M'

In [3]:
print(ds.dimensions)

{'time': <class 'netCDF4._netCDF4.Dimension'>: name = 'time', size = 2, 'lat': <class 'netCDF4._netCDF4.Dimension'>: name = 'lat', size = 361, 'lon': <class 'netCDF4._netCDF4.Dimension'>: name = 'lon', size = 576}


In [4]:
# print lowest and highest longitude and latitude
print(ds.variables['lon'][:].min(), ds.variables['lon'][:].max())
# print lowest and highest time
print(ds.variables['time'][:].min(), ds.variables['time'][:].max())

-180.0 179.375
38 39


In [5]:
from numpy.ma import masked

finalOutput = dict()

timeItems = ds.variables['time'][:]
latItems = ds.variables['lat'][:]
lonItems = ds.variables['lon'][:]

variableTargets = ["RH2M", "T2M", "PS", "PRECTOTCORR_SUM"]

for target in variableTargets:
    targetItems = ds.variables[target][:]
    for x in range(len(timeItems)):
        for y in range(len(latItems)):
            for z in range(len(lonItems)):
                val = targetItems[x][y][z]
                if val is masked:
                    continue

                lat = float(latItems[y])
                lon = float(lonItems[z])
                if lat not in finalOutput:
                    finalOutput[lat] = dict()
                if lon not in finalOutput[lat]:
                    finalOutput[lat][lon] = dict()
                if target not in finalOutput[lat][lon]:
                    finalOutput[lat][lon][target] = [0, 0]
                finalOutput[lat][lon][target][0] += val
                finalOutput[lat][lon][target][1] += 1
results = []
for lat in finalOutput:
    for lon in finalOutput[lat]:
        result = [lat, lon]
        for target in variableTargets:
            result.append(finalOutput[lat][lon][target][0] / finalOutput[lat][lon][target][1])
        results.append(result)
results.sort()
with open("meterology.csv", "w") as file:
    file.write(",".join(["lat", "lon"] + variableTargets))
    file.write("\n")
    for result in results:
        file.write(",".join([str(x) for x in result]))
        file.write("\n")            
